In [ ]:
%load_ext autoreload
%autoreload 2

In [16]:
from dishes_forecasting.predict.run_predict import Args
args = Args(
    company="RT",
    env="dev",
    is_running_on_databricks=False
)

In [17]:
from databricks_env import auto_setup_env
is_running_on_databricks = args.is_running_on_databricks
if is_running_on_databricks:
    auto_setup_env()

In [18]:
from constants.companies import get_company_by_code

company_code = args.company
company = get_company_by_code(company_code=company_code)
company_id = company.company_id

In [19]:
from dishes_forecasting.predict.configs import get_configs
pred_configs = get_configs(company_code=args.company)

In [20]:
from time_machine.forecasting import get_forecast_start

from datetime import date
if args.forecast_date is None:
    forecast_date = date.today()
year, week = get_forecast_start(
    cut_off_day=company.cut_off_week_day,
    forecast_date=forecast_date
)
min_yyyyww = year * 100 + week

In [ ]:
from dishes_forecasting.predict.data import create_pred_dataset
from dishes_forecasting.train.configs.feature_lookup_config import feature_lookup_config_list
df_pred, df_left = create_pred_dataset(
    schema='`~sylvia_liu_mlgold`',
    min_yyyyww=min_yyyyww,
    company=company,
    feature_lookup_config_list=feature_lookup_config_list
)

In [22]:
import mlflow
mlflow.set_registry_uri("databricks-uc")


In [ ]:
model_uri = pred_configs.model_uri[args.env]
loaded_model = mlflow.pyfunc.load_model(model_uri)


In [ ]:
from dishes_forecasting.predict.predictor import make_predictions
df_predictions = make_predictions(model=loaded_model, df_pred=df_pred)

In [ ]:
from dishes_forecasting.predict.postprocessor import postprocess_predictions
df_to_save = postprocess_predictions(
    df_predictions=df_predictions
)

df_to_save

In [26]:
from dishes_forecasting.predict.postprocessor import save_predictions
save_predictions(
    df_to_write=df_to_save,
    env=args.env,
    table_schema="mloutputs",
    table_name="ml_dishes_ratios"
)

In [ ]:

from dishes_forecasting.predict.run_predict import run_predict
df_predictions = run_predict(
    args=args,
)